In [1]:
import pandas as pd
import numpy as np
from ax_experiments_functions import rdc_loss_setup, rdc_read_glch_data, rdc_label_to_params
from ax_utils import gpei_method, build_experiment, initialize_experiment,initialize_experiment_with_trials
from ax_utils import get_min_list_from_df
from ax.modelbridge.registry import Models

In [2]:
data_csv_path = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/bpp-mse-psnr-loss-flops-params_bmshj2018"+\
"-factorized_10000-epochs_D-3-4_L-2e-2-1e-2-5e-3_N-32-64-96-128-160-192-224_M-32-64-96-128-160-192-224-256-288-320.csv"

In [3]:
lmbda="2e-2"
mult=100
weights=[1,float(lmbda)*(255**2),1/(1e6 * mult)]
lambdas=[lmbda]
complexity_axis="params"

In [4]:
seed= 1234
n_init = 6
n_batch=34

In [5]:
search_space, optimization_config = rdc_loss_setup(data_csv_path,weights,lambdas,complexity_axis)

In [14]:
gpei_min_list = gpei_method(search_space,optimization_config,seed,n_init,n_batch)

In [7]:
glch_csv_path = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/gho_results/"+\
"glch1D_weights_1e+00_1e+03_1e-08_lambdas_2e-2_bpp_loss_vs_mse_loss_vs_params_start_left_history.csv"

In [8]:
glch_df = rdc_read_glch_data(glch_csv_path)

In [11]:
glch_min_list = get_min_list_from_df(
    search_space,optimization_config,glch_df,rdc_label_to_params,["bpp_loss","mse_loss",complexity_axis],weights
)

In [13]:
pd.DataFrame({"glch_list": glch_min_list,
"gpei_list": gpei_min_list}).plot()